In [17]:
%%capture
%run 04_account.ipynb
%run 05_block.ipynb

## Miner

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes. Anyone with an account `acc` can become a miner. 

The miner takes a new block and tries to solve a specific computational puzzle, a specific hash. This can only be done by brute force. 

`coinbase` creates a tx to the miner as a reward for providing the pow (also called coinbase tx). 

In [79]:
class Miner(Hashable):
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def mine(self, txs, prev_header, diff, reward, attempts=1000):
        txs.insert(0, self.coinbase(reward))
        mt = MerkleTree(txs)
        bh = Header(mt.root, prev_header.hash, prev_header.number+1, len(txs))
        bh_b = bytes(bh)
        nonce = 0
        for i in range(attempts):
            candidate   = bh_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[2:2+diff] == '0'*diff: break
            nonce += 1
            assert nonce != attempts, 'no nonce could be found'
        bh.diff   = diff
        bh.reward = reward
        bh.nonce  = nonce
        bh.miner  = self.pub
        bh.mined  = True
        return Block(bh, txs)
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))

### Genesis Block

Is the first block in the blockchain. This is how it all begins. 

In [80]:
def mine_genesis(txs):
    mt  = MerkleTree(txs)
    bh  = Header(mt.root, rh(), 0, len(txs))
    return Block(bh, txs)

gb  = mine_genesis(r_stxs(2)); print(gb)

root:          🔀 0x9aceb893cd...10e
hash:          🔯 0xc9441d2ba6...cfa
prev_hash:     💮 0x487de97332...be1
number:        0
n_txs:         2
mined:         False
time:          Tue Mar 30 21:18:09 2021
volume:        14.0 ether
fees:          0.12 ether

txs:
🕡 0xfb3a737e98...8e4
🕅 0xdf58975055...c95


### Mining

Mine a block on top of the genesis block

In [84]:
miner = Miner(acc1)
mb = miner.mine(r_stxs(10), gb.bh, 1, 100); print(mb)

root:          🔀 0x9abb6ba0f0...d4a
hash:          💈 0x229507755c...557
prev_hash:     🔯 0xc9441d2ba6...cfa
number:        1
n_txs:         11
mined:         True
time:          Tue Mar 30 21:18:18 2021
diff:          1
reward:        100
nonce:         8
miner:         📢 0x7cA5161dCf...EA2
volume:        168.0 ether
fees:          0.62 ether

txs:
💨 0x42568bbc1d...f21
🔨 0xc2312babf1...137
📵 0x8f18d713b2...82d
💯 0x496a85dc72...33a
📻 0x954bd774e2...8fc
👾 0x188cab6e01...1de
🔑 0xab1872cb5c...7c2
👹 0x1349e7e157...271
💪 0x4434a3619a...0b6
💼 0x566e2f4311...9a6
💩 0x43d6da7ff7...7ed
